In [ ]:
from nilearn import image, plotting
import nibabel as nib
from nilearn.masking import apply_mask, compute_epi_mask
import matplotlib.pyplot as plt
import time



In [ ]:
file1 = "data/derivatives/derivatives-sub-CN001-func-sub-CN001_task-lppCN_run-04_space-MNIColin27_desc-preproc_bold.nii.gz"
img = image.smooth_img(file1, fwhm="fast")
plotting.plot_img(img)

In [ ]:

masked_img = compute_epi_mask(file1)
masked_data = apply_mask(file1, masked_img)

plt.plot(masked_data), plotting.plot_anat(file1), plotting.plot_stat_map(file1), plotting.plot_epi(file1)

In [ ]:
image = nib.load(file1)

In [ ]:
h = image.header
image.shape, h

In [ ]:
data = image.get_fdata()
data_f = image.get_data_dtype()
data.shape, type(data)

In [ ]:
for i in range(283):
    plt.imshow(data[:,:,70,i])
    plt.show()